# **CMPT 726/419 A3 Q4: Neural Networks in PyTorch**

Do not edit any cells until told to do so—the ones directly below should not be changed so you can access the required data and model for this problem.

In [1]:
from tqdm.notebook import tqdm
import csv

import torch
import torch.nn as nn
from torch.optim import Adam, SGD
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim.lr_scheduler import CosineAnnealingLR

from torchvision import transforms
from torchvision.datasets import CIFAR10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
class XYDataset(Dataset):
    """A basic dataset where the underlying data is a list of (x,y) tuples. Data
    returned from the dataset should be a (transform(x), y) tuple.
    Args:
    source      -- a list of (x,y) data samples
    transform   -- a torchvision.transforms transform
    """
    def __init__(self, source, transform=transforms.ToTensor()):
        super(XYDataset, self).__init__()
        self.source = source
        self.transform = transform

    def __len__(self): return len(self.source)
    
    def __getitem__(self, idx):
        x,y = self.source[idx]
        return self.transform(x), y

def build_dataset():
    """Returns the subset of the CIFAR-10 dataset containing only horses and
    deer, with the labels for each class modified to zero and one respectively.
    """
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(32, scale=(0.75, 1.0)),
        transforms.ToTensor()])

    data = CIFAR10(root=".", train=True, download=True)
    data = [(x, (0 if y == 2 else 1)) for x,y in data if y in {2, 5}]
    return XYDataset(data, transform=transform)

In [3]:
def generate_test_predictions(model):
    """Generates test predictions using [model]."""
    data_te = torch.load("cifar2_te.pt")
    loader = DataLoader(data_te, batch_size=128, num_workers=6, shuffle=False)
    preds = []
    with torch.no_grad():
        for x,_ in loader:
            fx = model(x.to(device))
            preds += (fx > .5).float().view(-1).cpu().tolist()

    with open("test_predictions.csv", "w+") as f:
        writer = csv.writer(f)
        writer.writerow(["Id", "Category"])
        for idx,p in enumerate(preds):
            writer.writerow([str(idx), str(int(p))])
    tqdm.write("Wrote model predictions to test_predictions.csv")

In [4]:
class NN(nn.Module):
    """A simple ConvNet for binary classification."""
    def __init__(self):
        super(NN, self).__init__()
        self.c1 = nn.Conv2d(3, 32, kernel_size=3)
        self.c2 = nn.Conv2d(32, 32, kernel_size=3)
        self.fc = nn.Linear(25088, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        bs = len(x)
        fx = self.c1(x)
        fx = self.relu(fx)
        fx = self.c2(fx)
        fx = self.relu(fx)
        fx = fx.view(bs, -1)
        fx = self.fc(fx)
        fx = self.sigmoid(fx)
        return fx.view(-1)


### ----- EDIT NO CODE ABOVE THIS CELL -----
### ----- EDIT CODE BENEATH THIS CELL -----





In [5]:
def validate(model, loader):
    """Returns a (acc_val, loss_val) tuple, where [acc_val] and [loss_val] are
    respectively the validation accuracy and loss of [model] on the data in
    [loader].

    Args:
    model   -- a model, already moved onto the GPU
    loader  -- a DataLoader over validation data
    """
    assert isinstance(model, NN)
    assert isinstance(loader, DataLoader)
    # loader = loader.float()
    acc_val, loss_val = 0, 0
    loss_fn = nn.BCELoss(reduction="mean")
    ##### YOUR CODE STARTS HERE (redefine [acc_val] and [loss_val] somewhere) ##
    total = 0
    correct = 0
    with torch.no_grad():
      for data in (loader):
        images, labels = data
        output = model(images)
        _, predicted = torch.max(output.data, 0)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        loss = loss_fn(output, labels.float())
        loss_val = loss.item()
        acc_val = 100*correct*(1.0) // total

    ##### YOUR CODE ENDS HERE   ################################################
    return acc_val, loss_val

In [6]:
def one_epoch(model, optimizer, loader):
    """Returns a (model, optimizer, avg_loss) tuple after training [model] on
    the data in [loader] for one epoch.

    Args:
    model       -- the neural network to train, already moved onto the GPU
    optimizer   -- the optimizer used to train [model]
    loader      -- a DataLoader with data to train [model] on

    Returns
    model       -- [model] after training for one epoch
    optimizer   -- the optimizer used to train [model]
    avg_loss    -- the average loss of [model] on each batch of [loader]
    """
    assert isinstance(model, NN)
    assert isinstance(optimizer, torch.optim.Optimizer)
    assert isinstance(loader, DataLoader)
    
    avg_loss = 0
    loss_fn = nn.BCELoss(reduction="mean")
    ##### YOUR CODE STARTS HERE (redefine [avg_loss] somewhere) ################
    
    loss_data = 0
    for data in (loader):
      images, labels = data
      optimizer.zero_grad()
      output = model(images)
      loss = loss_fn(output, labels.float())
      loss.backward()
      optimizer.step()
      loss_data = loss.item()
    
    avg_loss = loss_data/loader.batch_size
   
    ##### YOUR CODE ENDS HERE   ################################################
    return model, optimizer, avg_loss

Next, write a function to take a set of hyperparameters and return a `(model, acc_val)` tuple with the model having been trained with the hyperparameters and `acc_val` being the validation accuracy of the model after training.

Note the `**kwargs` in the function definition. This means that the function can take
any number of keyword arguments as input, and they will be accessible inside the function despite not being specified in the function definition, and they will be accessible within dictionary named `kwargs` inside the function.

_By passing in arguments this way, you can define whatever hyperparameters you want to use and pass them in!_

In [53]:
def train_and_validate(data_tr, data_val, **hyperparameters):
    """Returns a (model, acc_val) tuple where [model] is a neural
    network of the NN class trained with [hyperparameters] on [data_tr] and 
    validated on [data_val], and [acc_val] is the validation accuracy of the
    model after training.

    Args:
    data_tr         -- Dataset of training data
    data_val        -- Dataset of validation data
    hyperparameters -- kwarg dictionary of hyperparameters
    """
    assert isinstance(data_tr, torch.utils.data.Dataset)
    assert isinstance(data_val, torch.utils.data.Dataset)
    
    model, acc_val = NN(), 0
    ##### YOUR CODE STARTS HERE (redefine [model] and [acc_val] somewhere) #####
    epochs = 25
    data_loader1 = DataLoader(data_tr, batch_size = hyperparameters.get("batch_size")) 
    data_loader2 = DataLoader(data_val, batch_size = hyperparameters.get("batch_size"))
    optimizer = SGD(model.parameters(), lr = hyperparameters.get("lr"), 
                    momentum = hyperparameters.get("momentum"), 
                    weight_decay = hyperparameters.get("weight_decay"))
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.01)
    for i in tqdm(range(epochs)):
      model, optimizer, avg_loss = one_epoch(model, optimizer, data_loader1)
      acc_val, loss_val = validate(model, data_loader2)
      print("accuracy:", acc_val, "loss:", loss_val)
      scheduler.step()

    ##### YOUR CODE ENDS HERE   ################################################
    return model, acc_val

In [56]:
################################################################################
# Do hyperparameter search using train_and_validate(). You should call
# generate_test_predictions() on the model you eventually compute as best to 
# get test predictions to submit to Kaggle. You should also split off some of
# the 
data = build_dataset()  # You should use this data for training and validation
best_model = NN()       # You should at some point name the model you want to
                        # generate test predictions with `best_model`
                        
##### YOUR CODE STARTS HERE ####################################################

train_data, val_data = torch.utils.data.random_split(data, [8000, 2000])
parameters = {"batch_size": 4, "lr": 0.1, "momentum": 0.9, "weight_decay": 1e-6}
model, acc_val = train_and_validate(train_data, val_data, **parameters)

##### YOUR CODE ENDS HERE   ####################################################

Files already downloaded and verified


  0%|          | 0/25 [00:00<?, ?it/s]

accuracy: 51.0 loss: 0.704414427280426
accuracy: 51.0 loss: 0.6931476593017578
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 50.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817
accuracy: 51.0 loss: 0.6931479573249817


In [57]:
generate_test_predictions(best_model)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Wrote model predictions to test_predictions.csv
